# 🧜‍♂️ MermaidTrace 在线互动演示 (中文版)

欢迎体验 **MermaidTrace**！这是一个强大的 Python 逻辑可视化工具，能够将你的代码执行流程自动转化为清晰的 Mermaid 时序图。

在这个笔记本中，你无需安装任何本地环境，即可体验 MermaidTrace 的核心功能：
- ✅ **自动化**：只需装饰器，无需手动绘图。
- ✅ **深层追踪**：支持类、嵌套函数、异步调用。
- ✅ **异常捕获**：自动记录并可视化错误发生的位置。
- ✅ **智能折叠**：自动处理循环和重复调用。

## 1. 安装环境

首先，我们需要安装 `mermaid-trace` 库。

In [ ]:
%pip install mermaid-trace

## 2. 准备可视化工具

为了在 Colab 中直接看到生成的时序图，我们定义一个简单的辅助函数来渲染 Mermaid 代码。

In [ ]:
from IPython.display import display, HTML
import uuid

def render_mermaid(filename):
    with open(filename, "r") as f:
        content = f.read()
    
    # 为每个图表生成唯一 ID，确保在多次调用时不会冲突
    div_id = f"mermaid-{uuid.uuid4().hex[:8]}"
    
    # 使用 Mermaid.js 渲染 (v10+ 推荐方式)
    html = f"""
    <div id=\"{div_id}\" class=\"mermaid\">
    {content}
    </div>
    <script type=\"module\">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
        
        // 初始化配置
        mermaid.initialize({{ startOnLoad: false, theme: 'default' }});
        
        // 显式触发指定元素的渲染
        const element = document.getElementById('{div_id}');
        if (element) {{
            await mermaid.run({{
                nodes: [element]
            }});
        }}
    </script>
    """
    display(HTML(html))

## 3. 基础用法：简单的业务追踪

假设我们有一个下单流程：用户登录 -> 检查库存 -> 完成支付。

In [ ]:
from mermaid_trace import trace, configure_flow

# 配置输出文件
configure_flow("basic_demo.mmd", overwrite=True)

@trace(source="User", target="AuthService")
def login(username):
    return verify_db(username)

@trace(source="AuthService", target="Database")
def verify_db(username):
    return True

@trace(source="User", target="OrderSys")
def place_order(item_id):
    if login("admin"):
        check_stock(item_id)
        process_payment()

@trace(source="OrderSys", target="Inventory")
def check_stock(item_id):
    return True

@trace(source="OrderSys", target="PayGateway")
def process_payment():
    return "Success"

# 运行业务逻辑
place_order("iphone_15")

print("✅ 时序图已生成！正在渲染...")
render_mermaid("basic_demo.mmd")

## 4. 进阶用法：类方法批量追踪

对于复杂的类，你不需要为每个方法手动添加装饰器，可以使用 `@trace_class`。

In [ ]:
from mermaid_trace import trace_class

configure_flow("class_demo.mmd", overwrite=True)

@trace_class(target="PaymentService")
class PaymentProcessor:
    def validate_card(self, card_no):
        return True
    
    def charge(self, amount):
        self.validate_card("1234")
        return "Paid"

processor = PaymentProcessor()
processor.charge(100)

render_mermaid("class_demo.mmd")

## 5. 异常可视化

当代码发生报错时，MermaidTrace 会自动捕获并在图中标记出错误路径。

In [ ]:
configure_flow("error_demo.mmd", overwrite=True)

@trace(source="Client", target="API")
def api_call():
    dangerous_operation()

@trace(source="API", target="Server")
def dangerous_operation():
    raise ValueError("数据库连接超时！")

try:
    api_call()
except Exception as e:
    print(f"捕获到预期错误: {e}")

render_mermaid("error_demo.mmd")

## 6. 异步追踪

MermaidTrace 完美支持 `asyncio` 异步逻辑。

In [ ]:
import asyncio
from mermaid_trace import trace

configure_flow("async_demo.mmd", overwrite=True)

@trace(source="User", target="AsyncServer")
async def async_task():
    await asyncio.sleep(0.1)
    await sub_task()

@trace(source="AsyncServer", target="DB")
async def sub_task():
    return "Done"

# 在 Colab 中直接运行异步函数
await async_task()

render_mermaid("async_demo.mmd")

## 7. 总结

通过上面的演示，你可以看到 MermaidTrace 如何让原本隐藏在代码深处的逻辑变得触手可及。

如果你觉得有用，欢迎：
- ⭐ 在 GitHub 上点个 Star: [xt765/mermaid-trace](https://github.com/xt765/mermaid-trace)
- 📖 查看完整文档: [User Guide](https://github.com/xt765/mermaid-trace/blob/main/docs/zh/USER_GUIDE.md)